# 卷积在图像中的应用
:label:`sec_conv_layer`

现在我们已经理解了卷积层的理论工作原理，
接下来我们看看它们在实际中的运作。
基于我们将卷积神经网络视为探索图像数据结构的有效架构这一动机，
我们继续以图像作为贯穿始终的例子。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 互相关运算

回想一下，严格来说，卷积层
是一个误称，因为它们所表达的操作
更准确地说是互相关。
根据我们在:numref:`sec_why-conv`中对卷积层的描述，
在这样的层中，输入张量
和核张量结合
通过（**互相关运算**）产生输出张量。

现在我们先忽略通道，看看这如何在二维数据和隐藏表示中工作。
在:numref:`fig_correlation`中，
输入是一个高度为3、宽度为3的二维张量。
我们将张量的形状标记为$3 \times 3$或($3$, $3$)。
核的高度和宽度都是2。
*核窗口*（或*卷积窗口*）的形状由核的高度和宽度给出
（这里它是$2 \times 2$）。

![二维互相关运算。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素：$0\times0+1\times1+3\times2+4\times3=19$。](../img/correlation.svg)
:label:`fig_correlation`

在二维互相关运算中，
我们从位于输入张量左上角的卷积窗口开始
并在输入张量上滑动它，
从左到右，从上到下。
当卷积窗口滑动到某个位置时，
包含在该窗口中的输入子张量
和核张量逐元素相乘
并将结果张量求和
产生一个单一的标量值。
这个结果给出了输出张量
在相应位置的值。
这里，输出张量的高度为2，宽度为2
四个元素来自
二维互相关运算：

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$

请注意，在每个轴上，输出大小
略小于输入大小。
因为核的宽度和高度大于$1$，
我们只能正确地计算那些核完全位于图像内的位置的互相关，
输出大小由输入大小$n_\textrm{h} \times n_\textrm{w}$
减去卷积核大小$k_\textrm{h} \times k_\textrm{w}$
通过

$$(n_\textrm{h}-k_\textrm{h}+1) \times (n_\textrm{w}-k_\textrm{w}+1).$$

这是因为我们需要足够的空间
来“移动”卷积核穿过图像。
稍后我们将看到如何通过在图像边界周围填充零
保持大小不变
以确保有足够的空间移动核。
接下来，我们在`corr2d`函数中实现这一过程，
该函数接受输入张量`X`和核张量`K`
并返回输出张量`Y`。

In [2]:
def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

我们可以从 :numref:`fig_correlation` 构建输入张量 `X` 和核张量 `K`，以[**验证上述二维互相关操作实现的输出**]。

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 卷积层

卷积层将输入和核进行互相关运算，并加上一个标量偏置来产生输出。卷积层的两个参数是核和标量偏置。在基于卷积层训练模型时，我们通常会随机初始化核，就像全连接层一样。

我们现在准备好基于上面定义的 `corr2d` 函数[**实现一个二维卷积层**]。在 `__init__` 构造方法中，我们将 `weight` 和 `bias` 声明为两个模型参数。前向传播方法调用 `corr2d` 函数并加上偏置。

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

在
$h \times w$ 卷积
或 $h \imes w$ 卷积核中，
卷积核的高度和宽度分别为 $h$ 和 $w$。
我们还把
使用 $h \times w$
卷积核的卷积层简单称为 $h \times w$ 卷积层。

## 图像中的物体边缘检测

让我们花点时间解析[**卷积层的一个简单应用：
检测图像中物体的边缘**]
通过找到像素变化的位置。
首先，我们构建一个 $6\times 8$ 像素的“图像”。
中间四列是黑色（$0$），其余部分是白色（$1$）。

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构建一个高度为1、宽度为2的核`K`。
当我们对输入执行互相关操作时，
如果水平相邻的元素相同，
输出为0。否则，输出为非零。
请注意，这个核是有限差分算子的一个特例。在位置$(i,j)$它计算$x_{i,j} - x_{(i+1),j}$，即计算水平相邻像素值之间的差异。这是水平方向一阶导数的离散近似。毕竟，对于函数$f(i,j)$，其导数$-\partial_i f(i,j) = \lim_{\epsilon \to 0} \frac{f(i,j) - f(i+\epsilon,j)}{\epsilon}$。让我们看看这在实践中是如何工作的。

In [6]:
K = torch.tensor([[1.0, -1.0]])

我们已经准备好执行互相关运算，参数为 `X`（我们的输入）和 `K`（我们的核）。如你所见，[**我们检测从白到黑的边缘为 $1$，从黑到白的边缘为 $-1$。**] 所有其他输出值为 $0$。

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

我们现在可以将核应用于转置后的图像。
如预期的那样，它消失了。[**核 `K` 只能检测垂直边缘。**]

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习一个核

如果确切地知道我们需要的是什么，那么通过有限差分 `[1, -1]` 设计边缘检测器是很巧妙的。
然而，当我们查看更大的核，
并考虑卷积的连续层时，
手动精确指定每个滤波器应该做什么可能是不可能的。

现在让我们看看是否能够仅通过观察输入-输出对来[**学习从 `X` 生成 `Y` 的核**]。
我们首先构建一个卷积层，
并将它的核初始化为一个随机张量。
接下来，在每次迭代中，我们将使用平方误差
来比较 `Y` 和卷积层的输出。
然后我们可以计算梯度来更新核。
为了简单起见，
在下面的例子中，
我们使用内置的二维卷积层类
并忽略偏置。

In [9]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 16.481
epoch 4, loss 5.069
epoch 6, loss 1.794
epoch 8, loss 0.688
epoch 10, loss 0.274


请注意，经过10次迭代后，误差已经降到了一个很小的值。现在我们将[**看看我们学到的核张量。**]

In [10]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0398, -0.9328]])

确实，所学到的核张量与我们之前定义的核张量 `K` 非常接近。

## 互相关和卷积

回想我们在 :numref:`sec_why-conv` 中观察到的互相关和卷积操作之间的对应关系。这里我们继续考虑二维卷积层。如果这些层执行严格的卷积操作（如在:eqref:`eq_2d-conv-discrete`中定义）而不是互相关操作会怎样？为了获得严格卷积操作的输出，我们只需要将二维核张量水平和垂直翻转，然后与输入张量进行互相关操作。

值得注意的是，由于核是从数据中学习得到的，在深度学习中，无论这些层执行的是严格的卷积操作还是互相关操作，卷积层的输出都不会受到影响。

为了说明这一点，假设一个卷积层执行互相关并学习了 :numref:`fig_correlation` 中的核，这里用矩阵 $\mathbf{K}$ 表示。假设其他条件不变，当这个层改为执行严格的卷积时，学习到的核 $\mathbf{K}'$ 将是 $\mathbf{K}$ 在水平和垂直翻转后的结果。也就是说，当卷积层对 :numref:`fig_correlation` 中的输入和 $\mathbf{K}'$ 执行严格的卷积时，将得到与 :numref:`fig_correlation` 中相同的输出（即输入和 $\mathbf{K}$ 的互相关）。

遵循深度学习文献中的标准术语，我们将继续称互相关操作为卷积，尽管严格来说，它略有不同。此外，我们使用“元素”一词来指代任何表示层表示或卷积核的张量中的条目（或组件）。

## 特征图和感受野

如 :numref:`subsec_why-conv-channels` 所述，:numref:`fig_correlation` 中的卷积层输出有时被称为特征图，因为它可以被视为传递给后续层的空间维度（例如宽度和高度）上的学习表示（特征）。在CNN中，对于某一层中的任何元素 $x$，其感受野是指在前向传播过程中可能影响 $x$ 计算的所有元素（来自所有先前层）。请注意，感受野可能大于实际输入的大小。

让我们继续使用 :numref:`fig_correlation` 来解释感受野。给定 $2 \times 2$ 卷积核，值为 $19$ 的阴影输出元素的感受野是输入中阴影部分的四个元素。现在我们将 $2 \times 2$ 输出记为 $\mathbf{Y}$，并考虑一个更深的CNN，该网络有一个额外的 $2 \times 2$ 卷积层，以 $\mathbf{Y}$ 作为输入，输出单个元素 $z$。在这种情况下，$z$ 对 $\mathbf{Y}$ 的感受野包括 $\mathbf{Y}$ 的所有四个元素，而对输入的感受野则包括所有的九个输入元素。因此，当特征图中的任何元素需要更大的感受野来检测更广泛区域的输入特征时，我们可以构建一个更深的网络。

感受野这一名称源自神经生理学。一系列使用不同刺激物对各种动物进行的实验 :cite:`Hubel.Wiesel.1959,Hubel.Wiesel.1962,Hubel.Wiesel.1968` 探索了所谓的视觉皮层对这些刺激物的反应。总的来说，他们发现较低层次对边缘和相关形状作出反应。后来，:citet:`Field.1987` 使用可以称为卷积核的东西在自然图像上展示了这种效果。我们重印了 :numref:`field_visual` 中的关键图来展示惊人的相似性。

![取自 :citet:`Field.1987` 的图和标题：六个不同通道的编码示例。(左) 每个通道关联的六种类型传感器的例子。(右) (中间) 图像与(左)所示的六个传感器的卷积。单个传感器的响应是通过以与传感器大小成比例的距离采样这些过滤后的图像确定的（显示为点）。此图仅显示了偶数对称传感器的响应。](../img/field-visual.png)
:label:`field_visual`

事实证明，即使对于训练在图像分类任务上的网络的深层计算出的特征也是如此，例如 :citet:`Kuzovkin.Vicente.Petton.ea.2018` 所示。总之，卷积已被证明是计算机视觉领域极其强大的工具，无论是在生物学中还是在代码中。因此，它们预示着近期深度学习的成功并不令人惊讶（事后看来）。

## 总结

卷积层所需的核心计算是一个互相关操作。我们看到，只需一个简单的嵌套for循环即可计算其值。如果我们有多个输入和多个输出通道，我们实际上是在通道之间执行矩阵-矩阵运算。可以看到，计算是直接的，最重要的是高度局部化。这允许显著的硬件优化，许多最近的计算机视觉成果正是因为这一点才成为可能。毕竟，这意味着芯片设计者可以在优化卷积时投资于快速计算而非内存。虽然这可能不会导致其他应用的最佳设计，但它确实打开了普及且负担得起的计算机视觉的大门。

就卷积本身而言，它们可用于多种用途，例如检测边缘和线条、模糊图像或锐化图像。最重要的是，统计学家（或工程师）不需要发明合适的滤波器。相反，我们可以简单地从数据中学习它们。这用基于证据的统计取代了特征工程启发式方法。最后，而且非常令人愉快的是，这些滤波器不仅有利于构建深层网络，还与大脑中的感受野和特征图相对应。这使我们相信我们走在正确的道路上。

## 练习

1. 构建一个具有对角线边缘的图像 `X`。
    1. 如果将本节中的核 `K` 应用于它会发生什么？
    1. 如果转置 `X` 会发生什么？
    1. 如果转置 `K` 会发生什么？
1. 手动设计一些核。
    1. 给定方向向量 $\mathbf{v} = (v_1, v_2)$，推导一个检测正交于 $\mathbf{v}$ 的边缘（即方向 $(v_2, -v_1)$ 的边缘）的边缘检测核。
    1. 推导二阶导数的有限差分算子。与其相关的最小卷积核尺寸是多少？图像中的哪些结构对其响应最强？
    1. 你如何设计一个模糊核？为什么可能想要使用这样的核？
    1. 获得$d$阶导数所需的最小核尺寸是多少？
1. 当你尝试自动找到我们创建的 `Conv2D` 类的梯度时，你会看到什么样的错误消息？
1. 如何通过改变输入和核张量将互相关操作表示为矩阵乘法？

[讨论](https://discuss.d2l.ai/t/66)